In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 16:40:59.504956 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 16:40:59.511889 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 16:40:59.516489 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 16:40:59.522803 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 16:40:59.527221 140115358799680 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 16:40:59.535525 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 16:40:59.576791 140115358799680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
# NB_EPOCH = 200
# PATIENCE = 10  # early stopping patience
# # Helper variables for main training loop
# wait = 0
# preds = None
# best_val_loss = 99999

# # Fit
# for epoch in range(1, NB_EPOCH+1):

#     # Log wall-clock time
#     t = time.time()

#     # Single training iteration (we mask nodes without labels for loss calculation)
#     model.fit(graph, y_train, sample_weight=train_mask,
#               batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

#     # Predict on full dataset
#     preds = model.predict(graph, batch_size=A.shape[0])

#     # Train / validation scores
#     train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
#                                                    [idx_train, idx_val])
#     print("Epoch: {:04d}".format(epoch),
#           "train_loss= {:.4f}".format(train_val_loss[0]),
#           "train_acc= {:.4f}".format(train_val_acc[0]),
#           "val_loss= {:.4f}".format(train_val_loss[1]),
#           "val_acc= {:.4f}".format(train_val_acc[1]),
#           "time= {:.4f}".format(time.time() - t))

#     # Early stopping
#     if train_val_loss[1] < best_val_loss:
#         best_val_loss = train_val_loss[1]
#         wait = 0
#     else:
#         if wait >= PATIENCE:
#             print('Epoch {}: early stopping'.format(epoch))
#             break
#         wait += 1

In [13]:
# test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
# print(f"""
# loss = {test_loss[0]:.4f}
# accuracy = {test_acc[0]:.4f}
# """.strip())

# Gaussian Graph Convolution

In [14]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, A.shape[0], 
    is_first=True, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1], A.shape[0], 
    is_last=True,
    activation='softmax')(H+G)

W1110 16:40:59.633147 140115358799680 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:160: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 16:40:59.634472 140115358799680 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [15]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [16]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 16:40:59.719927 140115358799680 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9411 train_acc= 0.2857 val_loss= 1.9415 val_acc= 0.2333 time= 1.0831
Epoch: 0002 train_loss= 1.9382 train_acc= 0.2429 val_loss= 1.9357 val_acc= 0.3200 time= 0.0190
Epoch: 0003 train_loss= 1.9306 train_acc= 0.3500 val_loss= 1.9323 val_acc= 0.3000 time= 0.0234
Epoch: 0004 train_loss= 1.9222 train_acc= 0.3857 val_loss= 1.9236 val_acc= 0.3500 time= 0.0188
Epoch: 0005 train_loss= 1.9083 train_acc= 0.5000 val_loss= 1.9159 val_acc= 0.3733 time= 0.0190
Epoch: 0006 train_loss= 1.9019 train_acc= 0.4643 val_loss= 1.9080 val_acc= 0.4133 time= 0.0184
Epoch: 0007 train_loss= 1.8901 train_acc= 0.4786 val_loss= 1.8988 val_acc= 0.4467 time= 0.0184
Epoch: 0008 train_loss= 1.8781 train_acc= 0.5571 val_loss= 1.8912 val_acc= 0.4533 time= 0.0187
Epoch: 0009 train_loss= 1.8656 train_acc= 0.6000 val_loss= 1.8801 val_acc= 0.4733 time= 0.0235
Epoch: 0010 train_loss= 1.8505 train_acc= 0.6357 val_loss= 1.8697 val_acc= 0.5100 time= 0.0236
Epoch: 0011 train_loss= 1.8360 train_acc= 0.6357 v

Epoch: 0091 train_loss= 0.2762 train_acc= 0.9786 val_loss= 0.7020 val_acc= 0.8033 time= 0.0183
Epoch: 0092 train_loss= 0.2689 train_acc= 0.9786 val_loss= 0.6967 val_acc= 0.8067 time= 0.0208
Epoch: 0093 train_loss= 0.2615 train_acc= 0.9786 val_loss= 0.6910 val_acc= 0.8100 time= 0.0237
Epoch: 0094 train_loss= 0.2544 train_acc= 0.9786 val_loss= 0.6860 val_acc= 0.8100 time= 0.0180
Epoch: 0095 train_loss= 0.2471 train_acc= 0.9786 val_loss= 0.6811 val_acc= 0.8100 time= 0.0186
Epoch: 0096 train_loss= 0.2407 train_acc= 0.9786 val_loss= 0.6761 val_acc= 0.8100 time= 0.0229
Epoch: 0097 train_loss= 0.2343 train_acc= 0.9786 val_loss= 0.6718 val_acc= 0.8100 time= 0.0189
Epoch: 0098 train_loss= 0.2276 train_acc= 0.9786 val_loss= 0.6681 val_acc= 0.8133 time= 0.0228
Epoch: 0099 train_loss= 0.2215 train_acc= 0.9786 val_loss= 0.6642 val_acc= 0.8167 time= 0.0185
Epoch: 0100 train_loss= 0.2158 train_acc= 0.9786 val_loss= 0.6605 val_acc= 0.8200 time= 0.0186
Epoch: 0101 train_loss= 0.2095 train_acc= 0.9786 v

In [17]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6280
accuracy = 0.7990


# Attacks